In [1]:
from bs4 import BeautifulSoup
import requests
from collections import defaultdict

In [2]:
# Функция для получения содержимого страницы
def fetch_page(year):
    url = f"https://monocle.ru/reyting400/{year}/"
    response = requests.get(url)
    return response.text if response.status_code == 200 else None

# Функция для анализа HTML и извлечения нужной информации
def parse_html(html, year):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': 'rating-table auto'})
    
    headers = [th.get_text(strip=True) for th in table.find('thead').find_all('th')]
    #headers = headers[1:] if year == "2023" else headers

    rows = table.find('tbody').find_all('tr')
    rows = rows[1:] if year == "2023" else rows
    
    data = []
    for row in rows:
        cells = row.find_all('td')
        row_data = {headers[i]: cell.get_text(strip=True) for i, cell in enumerate(cells) if i < len(headers)}
        data.append(row_data)

    return data

def aggregate_data(years, region=None):
    all_data = []
    for year in years:
        html_content = fetch_page(year)
        if html_content:
            data = parse_html(html_content, year)
            all_data.extend(data)
    
    if region:
        # Фильтрация данных по региону, если он указан
        all_data = [d for d in all_data if d.get('Регион регистрации компании / головной компании') == region]
        
    return all_data

def calculate_total_profit_by_sector(data):
    sector_profit = defaultdict(float)
    for company_data in data:
        profit = company_data.get('Чистая прибыль', '0').replace('\xa0', '').replace(',', '.')
        sector = company_data['Отрасль']
        try:
            sector_profit[sector] += float(profit)
        except ValueError:
            pass
    return sector_profit

# Вызов функции для сбора данных
all_data = aggregate_data(["2021", "2022", "2023"])

# Подсчет прибыли по отраслям без фильтра по региону
total_profit = calculate_total_profit_by_sector(all_data)
most_profitable_sector = max(total_profit, key=total_profit.get)
print(f"Самая прибыльная отрасль за последние 3 года: {most_profitable_sector}")

# Подсчет прибыли по отраслям с фильтрацией по Пермскому краю
perm_region_data = aggregate_data(["2021", "2022", "2023"], region="Пермский край")
perm_region_profit = calculate_total_profit_by_sector(perm_region_data)
most_profitable_sector_perm = max(perm_region_profit, key=perm_region_profit.get)
print(f"Самая прибыльная отрасль в Пермском крае за последние 3 года: {most_profitable_sector_perm}")


Самая прибыльная отрасль за последние 3 года: нефтяная и нефтегазовая промышленность
Самая прибыльная отрасль в Пермском крае за последние 3 года: химическая и нефтехимическая промышленность


In [3]:
from collections import Counter

def filter_data_by_industry(data, industries):
    industry_data = defaultdict(list)
    for company_data in data:
        for industry in industries:
            if industry in company_data['Отрасль'].lower():
                region = company_data['Регион регистрации компании / головной компании']
                industry_data[industry].append(region)
    return industry_data

def rank_regions_by_industry(industry_data):
    ranked_data = {}
    for industry, regions in industry_data.items():
        region_counts = Counter(regions)
        ranked_regions = sorted(region_counts.items(), key=lambda x: x[1], reverse=True)
        ranked_data[industry] = ranked_regions
    return ranked_data

industries = ['металлургия', 'химическая', 'нефтегазовая']
industry_data = filter_data_by_industry(all_data, industries)
ranked_data = rank_regions_by_industry(industry_data)

# Выводим ранжированный список регионов по отраслям и годам
for year in ["2021", "2022", "2023"]:
    print(f"Ранжированный список регионов за {year}:")
    for industry, regions in ranked_data.items():
        print(f"{industry.capitalize()} отрасль:")
        for region, count in regions:
            print(f"{region}: {count} предприятий")
    print("\n")


Ранжированный список регионов за 2021:
Нефтегазовая отрасль:
Москва: 22 предприятий
Ямало-Ненецкий автономный округ: 6 предприятий
Иркутская область: 6 предприятий
Республика Татарстан: 5 предприятий
Ханты-Мансийский автономный округ: 4 предприятий
Сахалинская область: 3 предприятий
Томская область: 3 предприятий
Удмуртская Республика: 3 предприятий
Ростовская область: 3 предприятий
Краснодарский край: 3 предприятий
Московская область: 2 предприятий
Оренбургская область: 2 предприятий
Металлургия отрасль:
Москва: 25 предприятий
Свердловская область: 8 предприятий
Челябинская область: 8 предприятий
Кемеровская область: 5 предприятий
Красноярский край: 3 предприятий
Липецкая область: 3 предприятий
Самарская область: 3 предприятий
Краснодарский край: 2 предприятий
Химическая отрасль:
Москва: 17 предприятий
Республика Татарстан: 7 предприятий
Пермский край: 6 предприятий
Самарская область: 5 предприятий
Новгородская область: 3 предприятий
Республика Башкортостан: 3 предприятий
Кемеровская 

In [4]:
import pandas as pd

# Загрузка данных из файла
file_path = 'data-2021-structure-2.csv'
data = pd.read_csv(file_path)

# Проверка первых нескольких строк данных, чтобы понять их структуру
data.head()

,Название региона или Федерального округа,Период,"Число музеев,филиалов и структурных подразделений, ед.","Число музеев, ед.","Число филиалов, ед.","Число структурных подразделений, ед.","из общего числа музеев имеют музейные предметы, внесенных в АМС, ед.",из общего числа музеев имеют предметы в цифровом изображении,"из общего числа музеев имеют предметы в цифровом изображении, доступные в Интернете \r\nна различных сайтах/порталах","из общего числа музеев имеют музейные предметы, внесенные в электронный каталог",...,"из израсходованных за год финансовых средств, тыс. руб. на пополнение фонда музея всего","из израсходованных за год финансовых средств, тыс. руб. на пополнение фонда музея из них \r\nза счет средств, полученных от предпринимательской деятельности и сдачи имущества в аренду\r\n","из израсходованных за год финансовых средств, тыс. руб. на организацию и проведение выставок всего","из израсходованных за год финансовых средств, тыс. руб. на организацию и проведение выставок из них \r\nза счет средств, полученных от предпринимательской деятельности и сдачи имущества в аренду\r\n","из израсходованных за год финансовых средств, тыс. руб. на реставрацию предметов основного и научно-вспомогательного фондов всего","из израсходованных за год финансовых средств, тыс. руб. на реставрацию предметов основного и научно-вспомогательного фондов из них \r\nза счет средств, полученных от предпринимательской деятельности и сдачи имущества в аренду\r\n","из израсходованных за год финансовых средств, тыс. руб. на информатизацию музейной деятельности, в т.ч. создание электронных каталогов и оцифровку музейного фонда всего","из израсходованных за год финансовых средств, тыс. руб. на информатизацию музейной деятельности, в т.ч. создание электронных каталогов и оцифровку музейного фонда из них \r\nза счет средств, полученных от предпринимательской деятельности и сдачи имущества в аренду\r\n","из израсходованных за год финансовых средств, тыс. руб. на обеспечение безопасности экспозиции, фондохранилищ, посетителей всего","из израсходованных за год финансовых средств, тыс. руб. на обеспечение безопасности экспозиции, фондохранилищ, посетителей из них \r\nза счет средств, полученных от предпринимательской деятельности и сдачи имущества в аренду\r\n"
0,Всего по Российской Федерации,2021,2981,1882,736.0,363,1607,1485,1322,2186,...,429703,147065,3626486,908488,563560,34290,459163,149509,3538212,428896
1,Центральный федеральный округ,2021,721,442,211.0,68,399,390,347,547,...,123849,30226,2167269,432041,126808,12281,245112,67629,1640763,228694
2,Белгородская обл.,2021,45,31,12.0,2,38,37,36,40,...,7784,609,6826,528,800,97,569,157,15675,1541
3,Брянская обл.,2021,33,17,14.0,2,15,13,12,29,...,332,232,4571,2047,545,0,43,0,6817,214
4,Владимирская обл.,2021,24,24,NaN,0,13,13,12,16,...,5763,603,12064,4467,1858,67,4993,316,36435,2257


In [5]:
# Удаление лишних пробелов в названиях столбцов
data.columns = [col.strip() for col in data.columns]

In [6]:
# print(data.columns.tolist())

In [7]:
grouped_data = data.groupby('Название региона или Федерального округа')['Число музеев, ед.'].sum()

In [8]:
# Сортировка результатов по убыванию
grouped_data = grouped_data.sort_values(ascending=False)

# Выведите результаты
print(grouped_data[:10])

Название региона или Федерального округа
Всего по Российской Федерации        1882
Центральный федеральный округ         442
Приволжский федеральный округ         421
Сибирский федеральный округ           238
Северо-Западный федеральный округ     194
Южный федеральный округ               187
Дальневосточный федеральный округ     185
Уральский федеральный округ           147
Московская обл.                        75
г.Москва                               74
Name: Число музеев, ед., dtype: int64
